# CODA-database-sandbox

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Date: 11 September 2024

# detect compute platform

In [1]:
import os
import socket
    
def get_platform():
    hostname = socket.gethostname()
    if "gadi" in hostname:  # Adjust this condition to fit your HPC's hostname or unique identifier
        return 'HPC',hostname
    else:
        return 'Laptop',hostname
[platform,hostname] = get_platform()
print('the platform we are working on is '+platform+' with hostname: '+hostname)

the platform we are working on is HPC with hostname: gadi-cpu-bdw-0122.gadi.nci.org.au


### import packages

In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43917,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:43543,Total threads: 4
Dashboard: /proxy/33895/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:33633,


In [4]:
if platform == 'HPC':
    data_path = '/g/data/es60/users/thomas_moore/CODA/2005/'
    write_path = '/g/data/es60/users/thomas_moore/CODA/parquet/'
else:
    data_path = '/Users/moo270/data/CARSv2/CODA/'
    write_path = '/Users/moo270/data/CARSv2/CODA/parquet/'

print(f"Using data path: {data_path}")
print(f"Using write path: {write_path}")

Using data path: /g/data/es60/users/thomas_moore/CODA/2005/
Using write path: /g/data/es60/users/thomas_moore/CODA/parquet/


# build PQ & CSV versions from NetCDF

In [10]:
ds = xr.open_dataset(data_path+ "WOD_CODA_2005_pfl.nc",chunks="auto")

In [11]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates:
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
Data variables: (12/29)
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    country                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    dataset                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Access_no               (cast) float64 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    Recorder                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    crs                     int32 4B ...
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

In [14]:
ds = ds.set_coords(['WOD_id',
                    'origflagset',
                    'country',
                    'dataset',
                    'Access_no',
                    'dbase_orig',
                    'Project',
                    'WOD_cruise_identifier',
                    'Institute',
                    'Ocean_Vehicle',
                    'Temperature_Instrument',
                    'CODA_id',
                    'Recorder',
                    'Platform',
                    'crs'])

In [15]:
ds

<xarray.Dataset> Size: 7GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates: (12/21)
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Ocean_Vehicle           (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Temperature_Instrument  (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    crs                     int32 4B ...
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Data variables: (12/14)
    Temperature             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Temperature_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Temperature_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Salinity                (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Salinity_WODflag        (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Salinity_origflag       (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    ...                      ...
    Oxygen_origflag         (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

# make truncated version of ds

In [9]:
ds_truncated = ds#.isel(cast=slice(0,10000))

In [ ]:
%%time
df = ds_truncated[['Temperature','Salinity']].to_dataframe().reset_index()
df

# Try removing all-column-NaN-rows here? - https://github.com/CARSv2/cars-v2/issues/26

In [ ]:
%%time
df1 = df.dropna(subset=['Temperature', 'z']).reset_index(drop=True)

In [ ]:
df1

In [ ]:
%%time
df2 = df.dropna(subset=['z']).reset_index(drop=True)

In [ ]:
df2

In [ ]:
%%time
df3 = df.dropna(subset=['Temperature']).reset_index(drop=True)

In [ ]:
df3

# FIX THIS ???? go from DS - DDF lazily

# convert to dask dataframe

In [ ]:
%%time
ddf = dd.from_pandas(df, npartitions=500)

In [ ]:
ddf

# find columns with byte strings and convert ( is this needed when going from ds --> df --> ddf )

In [ ]:
ddf.head()

In [ ]:
%%time
ddf.info(memory_usage='deep')

In [ ]:
%%time
ddf.to_parquet(write_path+"2005_pfl_temp.pq",engine='pyarrow', compression='snappy')

In [ ]:
%%time
df.to_csv(write_path+"2005_pfl_temp.csv", index=False)

# load parquet data & filter

In [ ]:
%%time
temp_pfl_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")
# Search for rows where 'column_name' matches a condition
result_parquet = temp_pfl_ddf[temp_pfl_ddf ['z'] >= 2000].compute()

In [ ]:
result_parquet

# load CSV data and filter

In [ ]:
%%time
temp_pfl_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks
# Search for rows where 'column_name' matches a condition
result_csv = temp_pfl_csv_ddf[temp_pfl_csv_ddf['z'] >= 2000].compute()

# lazy load temperature

In [ ]:
%%time
temp_pq_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")

In [ ]:
%%time
temp_csv_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks

In [ ]:
%%time
temp_csv_ddf[temp_csv_ddf['z'] >= 2000].compute()

In [ ]:
%%time
temp_pq_ddf[temp_pq_ddf['z'] >= 2000].compute()

## Polars

In [ ]:
import polars as pl

In [ ]:
%%time
# Load a large dataset with Dask (e.g., CSV)
dask_df = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)

# Convert Dask DataFrame to Pandas
pandas_df = dask_df.compute()

# Convert Pandas DataFrame to Polars
polars_df = pl.from_pandas(pandas_df)

In [ ]:
%%time
polars_df[polars_df['z'] >= 2000].compute()

# list of todo's

- create parquet from a df
- consider utility of turning "metadata" vars into coords
- consider removing NaN's at df step
- consider removing NaN's otherwise



In [ ]:
df.dropna(how='all',column='z')

In [ ]:
%%time
df = ds.to_dataframe()#.reset_index()